In [6]:
import pandas as pd
import matplotlib.pyplot as plt
import amplpy
from pathlib import Path
import os
import shutil

from util.return_up_outliers_func import *

In [7]:
# Path for the run
path_unit_tests = Path(r'..\..\unit_tests')
path_data_network = Path(r'.\data_network')
path_ampl_results = Path(r'.\results')

In [8]:
def eval_level_of_correctness_buses(df_buses, expected, results, verbose=False):
    """ Eval the level of detection for buses in opti results.
    
    df_buses : pandas.dataframe containing network buses information.
    expected : the substation expected.
    results : the buses detected with an error.
    
    If one the buses of results is in the substation expected, return 1. Otherwise, return -1.
    """

    df_substation = pd.read_table(path_data_network / 'ampl_network_substations.txt', sep=" ", header=1).drop(columns=["#\"variant\""])
    num_substation_expected = df_substation[df_substation["id"] == expected]["num"].to_numpy()[0]

    for bus in results:
        if df_buses[df_buses["id"] == bus]["substation"].to_numpy()[0] == num_substation_expected :
            return 1

    return -1

def eval_level_of_correctness_branches(df_branches, expected, results, verbose=False):
    """ Eval the level of detection for branches in opti results. 

    df_branches : pandas.dataframe containing information on branches of the network.
    expected : the branche expected.
    results : the branches detected with an error.

    If expected is in results, return 1.
    Else if expected share a common bus with one branch of results, return 0.
    Else, return -1."""
    
    if expected in results:
        return 1
    
    elif len(df_branches[df_branches['id'] == expected]['bus1'].to_numpy()) != 0 and len(df_branches[df_branches['id'] == expected]['bus2'].to_numpy()) != 0:
        bus1_exp = df_branches[df_branches['id'] == expected]['bus1'].to_numpy()[0]
        bus2_exp = df_branches[df_branches['id'] == expected]['bus2'].to_numpy()[0]

        for result in results:

            buses_result = [df_branches[df_branches['id'] == result]['bus1'].to_numpy()[0], df_branches[df_branches['id'] == result]['bus2'].to_numpy()[0]]
            if bus1_exp in buses_result or bus2_exp in buses_result:
                return 0 # FIXME ICI BIZARRE QU'ON AIT JAMAIS 0.

    return -1

def get_penalized_buses(df_results, threshold):
    """ Return the buses that are penalized. A penalized bus is a bus with a high sigma value on it."""
    buses_detected = set()
    df_data = pd.read_table(path_data_network / 'ampl_network_buses.txt', sep=" ", header=1).drop(columns=["#\"variant\""])

    for index, _ in return_up_outliers_Zscore(df_results["s1"].abs(), threshold).items():
        buses_detected.add(df_data['id'].get(df_results['busPV'].get(index) - 1))

    return buses_detected

def get_penalized_branches(df_results, threshold):
    """ Return the branches that are penalized. A penalized branch is a branch with a high sigma value on it."""

    branches_detected = set()
    df_data = pd.read_table(path_data_network / 'ampl_network_branches.txt', sep=" ", header=1).drop(columns=["#\"variant\""])
    
    for i in range(1,9):
        for index, _ in return_up_outliers_Zscore(df_results["sigma"+str(i)].abs(), threshold).items():
            branches_detected.add(df_data['id'].get(df_results['branch'].get(index) - 1))

    return branches_detected


In [9]:
threshold = 3

expected = {}
results = {}
scores = {}

for test_category in os.listdir(path_unit_tests):
    if test_category == "targetV":
        continue

    expected[test_category] = {}
    results[test_category] = {}
    scores[test_category] = {}

    for unit_test in os.listdir(path_unit_tests / str(test_category)):

        path_unit_test = Path(path_unit_tests /test_category / unit_test)

        # filter tests with no expected results
        if "results_expected.txt" not in os.listdir(path_unit_test):
            continue

        # Here we know that there is 2 lines because we expect only one branch with an error
        with open(path_unit_test / "results_expected.txt", 'r') as file:
            file.readline() # Skip the comment on first line
            expected[test_category][unit_test] = file.readline().replace('\n', '')

        # Copy the network files in corresponding directory
        for file in os.listdir(path_unit_test):
            shutil.copyfile(path_unit_test / file, path_data_network / file)

        # Execute the ampl file corresponding to error detection model
        ampl = amplpy.AMPL()
        ampl.read('detection.run') # So we export results files in path_ampl_results
        ampl.close()

        # Read results after running

        # TODO : Add check for target V tests
        #if test_category == "targetV":
        #    df_results_s = pd.read_csv(path_ampl_results / "s1_penal.csv", header=1, sep=";") # TODO : Be careful here, path is absolute
        #    df_buses = pd.read_table(path_data_network / 'ampl_network_buses.txt', sep=" ", header=1).drop(columns=["#\"variant\""])
        #    results[test_category][unit_test] = get_penalized_buses(df_results_s, threshold)
#
        #    print(test_category)
        #    print(unit_test)
        #    scores[test_category][unit_test] = eval_level_of_correctness_buses(df_results_s, expected[test_category][unit_test], results[test_category][unit_test]) 
        #else:
        df_results_sigma = pd.read_csv(path_ampl_results / "sigma_penal_dbp.csv", header=1, sep=";") # TODO : Be careful here, path is absolute
        results[test_category][unit_test] = get_penalized_branches(df_results_sigma, threshold)
        scores[test_category][unit_test] = eval_level_of_correctness_branches(df_results_sigma, expected[test_category][unit_test], results[test_category][unit_test])
        
        print("All expected = ", expected)
        print("All results =", results)

        print("For test " + test_category + ", " + unit_test + " : ")
        print("Score of current results : ", scores[test_category][unit_test])
        print("Number of elements in current results : ", len(results[test_category][unit_test]))






















*** Start of file divergence analysis : Tue May 23 11:00:46 2023
Parameter: threshold to decide wether an active or reactive power value is zero Pnull:=0.01 (MW or Mvar or MVA)
Parameter: threshold to detect zero impedance branch Znull:=0.0001 pu
Parameter: for consistency checks of minimum nominal voltages epsilon_nominal_voltage:= 1 kV
Parameter: for consistency checks of voltage bounds eps<=Vmin<Vmax<=2-eps, epsilon_min_voltage:= 0.5 pu
Parameter: maximum for generating units parameters Pmin Pmax Qmin Qmax = 9000 MW or Mvar
Parameter: defaultPmax = 1000 MW
Parameter: defaultPmin = 0 MW
Parameter: defaultQmaxPmaxRatio = 0.3 Mvar/MW
Parameter: defaultQmin = -300 Mvar
Parameter: defaultQmax = 300 Mvar
Parameter: minimalQPrange = 1 MW or Mvar

*** Connexity computation
# CCcomp solve: start (Tue May 23 11:00:46 2023)

Artelys Knitro 14.0.0: outlev=0
Knitro 14.0.0: Locally optimal or satisfactory solution.
objective -2.797762022e-14; feasibility error 0
0 iterations; 

In [ ]:
global_scores = {}
for param in scores.keys():
    global_scores[param] = 0

    for test in scores[param].keys():
        print("Number of elements for " + param + " " + test + " : " + str(len(results[param][test])))
        print(results[param][test]) 
        if scores[param][test] > -1:
            global_scores[param] += 1

        #print("For param " + param + " and test " + test + ", score = " + str(scores[param][test]))
        #print("Expected branches was : " + expected[param][test])
        #print("Results gotten : " + str(results[param][test]) + "\n")

# Resume
print()
for param in global_scores.keys():
    print("For parameter " + param + " :")
    print("Number of tests passed : " + str(global_scores[param]) + " / " + str(len(results[param])))
    print("Number of tests wrong : " + str(len(results[param]) - global_scores[param]) + " / " + str(len(results[param])))

print(scores)

Number of elements for alpha test0 : 1
{'.TDVAL72.VANY'}
Number of elements for alpha test1 : 1
{'.TDVAL72.VANY'}
Number of elements for alpha test10 : 3
{'.TDVAL72.VANY', 'BEZAUL61VANDI', 'BEZAUL62VANDI'}
Number of elements for alpha test11 : 1
{'.TDVAL72.VANY'}
Number of elements for alpha test2 : 1
{'.TDVAL72.VANY'}
Number of elements for alpha test3 : 1
{'.TDVAL72.VANY'}
Number of elements for alpha test4 : 1
{'.TDVAL72.VANY'}
Number of elements for alpha test5 : 1
{'.TDVAL72.VANY'}
Number of elements for alpha test6 : 1
{'.TDVAL72.VANY'}
Number of elements for alpha test7 : 1
{'.TDVAL72.VANY'}
Number of elements for alpha test8 : 3
{'.TDVAL72.VANY', 'BEZAUL61VANDI', 'BEZAUL62VANDI'}
Number of elements for alpha test9 : 1
{'.TDVAL72.VANY'}
Number of elements for B1 test0 : 1
{'HELLEL61ZHEL5'}
Number of elements for B1 test1 : 1
{'BLOCAL41BZBEL'}
Number of elements for B1 test10 : 1
{'.TDVAL74.VANY'}
Number of elements for B1 test11 : 2
{'MAZURY772', 'NEUF5L42ZAUMA'}
Number of eleme